In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [9]:
import pandas as pd
import pickle

sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
sales_data = sales.iloc[:, 6:].T  # Shape: (days, items) → (1913, 30490)
sales_data.columns = sales['id'].values

# === Load calendar.csv ===
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")

# === Select calendar columns used in training ===
calendar_cols = [
    'wday', 'month', 'year',
    'event_name_1', 'event_name_2', 'event_type_1',
    'snap_CA', 'snap_TX', 'snap_WI'
]

# Prepare full calendar_feats first (used for training)
calendar_train = calendar.iloc[:1913][calendar_cols].copy()
calendar_train = calendar_train.fillna("none")
calendar_feats = pd.get_dummies(calendar_train, columns=['event_name_1', 'event_name_2', 'event_type_1'])


# ✅ Save calendar feature columns for future alignment (important for validation/eval)
with open("calendar_columns.pkl", "wb") as f:
    pickle.dump(calendar_feats.columns.tolist(), f)


# Fit scaler on sales only
sales_scaler = MinMaxScaler()
sales_only_scaled = sales_scaler.fit_transform(sales_data)


combined = pd.concat([sales_data.reset_index(drop=True), calendar_feats], axis=1)


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined)

In [6]:
import pandas as pd
import numpy as np
import pickle
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# === Define sequence length ===
input_window = 28  # number of past days used as input
num_items = 30490  # number of items (sales columns only)
total_days = scaled_data.shape[0]  # total time steps available

# === Generate training sequences ===
X_train, y_train = [], []

for i in range(input_window, total_days):
    input_seq = scaled_data[i - input_window:i]  # shape: (28, features)
    target = scaled_data[i][:num_items]          # shape: (30490,), only sales part

    X_train.append(input_seq)
    y_train.append(target)

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

print(f"✅ X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

✅ X_train shape: (1885, 28, 30537), y_train shape: (1885, 30490)


In [11]:
# === Extract next 28 days: day 1914 to 1941 (for validation) or 1942–1969 (for evaluation) ===
calendar_future = calendar.iloc[1913:1941][calendar_cols].copy()

# === Fill missing values and one-hot encode event-related categorical features ===
calendar_future = calendar_future.fillna("none")
calendar_future = pd.get_dummies(calendar_future, columns=['event_name_1', 'event_name_2', 'event_type_1'])

# === Ensure column alignment with training ===
calendar_future = calendar_future.reindex(columns=calendar_feats.columns, fill_value=0)

# === Convert to float32 NumPy array ===
calendar_next_28 = calendar_future.values.astype(np.float32)  # shape: (28, calendar_features)


# === Define Model Architecture ===
def build_global_lstm_model(input_timesteps, num_features, num_items, dropout_rate=0.3):
    model = Sequential()

    # LSTM layers
    model.add(LSTM(units=64, return_sequences=True, input_shape=(input_timesteps, num_features)))
    model.add(Dropout(dropout_rate))

    model.add(LSTM(units=96, return_sequences=True))
    model.add(Dropout(dropout_rate))

    model.add(LSTM(units=96))  # Last layer: return_sequences=False
    model.add(Dropout(dropout_rate))

    # Output layer (no activation)
    model.add(Dense(units=num_items))

    # Optimizer
    optimizer = AdamW(learning_rate=0.0005, weight_decay=1e-5)
    model.compile(optimizer=optimizer, loss='huber')

    # Learning rate scheduler
    lr_callback = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )

    return model, lr_callback

# === Build and Train Model ===
model, lr_callback = build_global_lstm_model(
    input_timesteps=28,
    num_features=X_train.shape[2],
    num_items=30490
)

# === Early stopping callback ===
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[lr_callback, early_stop],
    verbose=1
)

# === Load calendar columns for alignment ===
with open("calendar_columns.pkl", "rb") as f:
    calendar_columns = pickle.load(f)

calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
calendar_cols = ['wday', 'month', 'year', 'event_name_1', 'event_name_2', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI']

# === Prepare input and calendar features for validation ===
calendar_future_val = calendar.iloc[1913:1941][calendar_cols].copy()
calendar_future_val = calendar_future_val.fillna("none")
calendar_future_val = pd.get_dummies(calendar_future_val, columns=['event_name_1', 'event_name_2', 'event_type_1'])
calendar_future_val = calendar_future_val.reindex(columns=calendar_columns, fill_value=0)
calendar_next_28_val = calendar_future_val.values.astype(np.float32)

last_28_days_input_val = scaled_data[-28:, :].astype(np.float32)

# === Forecasting Function ===
def forecast_28_days(model, input_sequence, calendar_future, num_items):
    window = input_sequence.shape[0]
    input_data = input_sequence.copy()
    predictions = []

    for day in range(28):
        x_input = input_data[-window:].reshape(1, window, -1)
        y_pred = model.predict(x_input, verbose=0)

        # Debug prediction stats
        print("Prediction summary:", y_pred.min(), y_pred.max(), y_pred.mean())

        # Inverse scale only the sales part
        y_pred_original = sales_scaler.inverse_transform(y_pred)

        # Re-scale prediction to feed back into LSTM input (not using scaler on full combined)
        y_pred_scaled = sales_scaler.transform(y_pred)

        # Combine with correct day’s calendar feature
        next_input_row = np.concatenate([y_pred_scaled[0], calendar_future[day]])
        input_data = np.vstack([input_data, next_input_row])

        predictions.append(y_pred_original)

    return np.vstack(predictions)

# === Forecast Validation Period ===
forecast_val = forecast_28_days(
    model=model,
    input_sequence=last_28_days_input_val,
    calendar_future=calendar_next_28_val,
    num_items=30490
)
forecast_val = np.clip(forecast_val, 0, None).T

# === Build submission DataFrame for validation ===
sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
ids_val = sales['id'].values
submission_val_df = pd.DataFrame(forecast_val, columns=[f"F{i}" for i in range(1, 29)])
submission_val_df.insert(0, "id", ids_val)
submission_val_df.to_csv("submission_val.csv", index=False)
print("✅ submission_val.csv saved successfully!")

# === Prepare calendar for evaluation period ===
calendar_future_eval = calendar.iloc[1941:1969][calendar_cols].copy()
calendar_future_eval = calendar_future_eval.fillna("none")
calendar_future_eval = pd.get_dummies(calendar_future_eval, columns=['event_name_1', 'event_name_2', 'event_type_1'])
calendar_future_eval = calendar_future_eval.reindex(columns=calendar_columns, fill_value=0)
calendar_next_28_eval = calendar_future_eval.values.astype(np.float32)

last_28_days_input_eval = scaled_data[-28:, :].astype(np.float32)

# === Forecast Evaluation Period ===
forecast_eval = forecast_28_days(model, last_28_days_input_eval, calendar_next_28_eval, num_items=30490)
forecast_eval = np.clip(forecast_eval, 0, None).T

# === Build submission DataFrame for evaluation ===
ids_eval = [i.replace("validation", "evaluation") for i in sales["id"].values]
submission_eval_df = pd.DataFrame(forecast_eval, columns=[f"F{i}" for i in range(1, 29)])
submission_eval_df.insert(0, "id", ids_eval)
submission_eval_df.to_csv("submission_eval.csv", index=False)
print("✅ submission_eval.csv saved successfully!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - loss: 0.0092 - val_loss: 0.0103 - learning_rate: 5.0000e-04
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0071 - val_loss: 0.0098 - learning_rate: 5.0000e-04
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0067 - val_loss: 0.0096 - learning_rate: 5.0000e-04
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0065 - val_loss: 0.0095 - learning_rate: 5.0000e-04
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - loss: 0.0064 - val_loss: 0.0094 - learning_rate: 5.0000e-04
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - loss: 0.0063 - val_loss: 0.0093 - learning_rate: 5.0000e-04
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - loss: 0.0063 - val_loss: 0.0093 - learning_rate: 5.0000e-04
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0062
Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.0062 - val_loss

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction summary: -0.005227901 0.5124495 0.08387786
Prediction summary: -0.006550329 0.51327217 0.08373936
Prediction summary: -0.008458529 0.51282835 0.08347894
Prediction summary: -0.010415155 0.51170784 0.0831556


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


...

Prediction summary: -0.020944051 0.50304043 0.08088645
Prediction summary: -0.021159481 0.50303376 0.080824636
Prediction summary: -0.021369163 0.50284904 0.08073726
Prediction summary: -0.021536693 0.50265414 0.08065871


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction summary: -0.021653276 0.50245005 0.08058407
Prediction summary: -0.02167349 0.50206876 0.08050049
Prediction summary: -0.0215475 0.50099874 0.08035718
Prediction summary: -0.02106595 0.49743712 0.07997025


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Prediction summary: -0.019147813 0.48322758 0.07842721
✅ submission_eval.csv saved successfully!


In [12]:
import pandas as pd

# === Load validation and evaluation prediction files ===
submission_val = pd.read_csv("submission_val.csv")
submission_eval = pd.read_csv("submission_eval.csv")

# === Combine both parts ===
submission_all = pd.concat([submission_val, submission_eval], axis=0).reset_index(drop=True)

# === Load sample_submission to match required order ===
sample = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")

# === Align predictions to sample_submission id order ===
final_submission = sample[['id']].merge(submission_all, on='id', how='left')

# === Validate submission ===
num_duplicates = final_submission['id'].duplicated().sum()
num_missing = final_submission.isnull().sum().sum()

assert num_duplicates == 0, f"❌ Found {num_duplicates} duplicate ID(s) in submission"
assert num_missing == 0, f"❌ Found {num_missing} missing prediction(s) in submission"

# === Save final file ===
final_submission.to_csv("final_submission.csv", index=False)
print(f"✅ Final submission saved: {len(final_submission)} rows → final_submission.csv")

✅ Final submission saved: 60980 rows → final_submission.csv
